<a href="https://colab.research.google.com/github/Raginii/Color-Concept-Associations-using-Google-Images/blob/master/SphericalBest3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
ls

dataTop50.xlsx  drive/  sample_data/


In [0]:
import glob
from skimage.transform import resize
from skimage.io import imread, imsave, imshow
import os
import pandas as pd
import numpy as np
import skimage.color as skcolor
import matplotlib.pyplot as plt

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

Number of processors:  2


In [0]:
colNames = ["col_"+str(i) for i in range(58)] 
allConcepts = ['Mango','Watermelon','Honeydew','Cantaloupe','Grapefruit','Strawberry','Raspberry','Blueberry','Avocado','Orange','Lime','Lemon']

# Read data
# Get the color ratings data -From file data.csv
# data = pd.read_csv("data.csv",names = colNames)

# # Get the LAB coordinates of colors from file Lab.csv

# colorData = pd.read_csv("Lab.csv",header=None)
# colorDataRGB = pd.read_csv("RGB.csv",header=None)

#path = "E:\GitRepo\sampleLAB\Google Images ConceptAssociation/downloads"
#path = "E:/GitRepo/sampleLAB/Google Images ConceptAssociation/downloads"
#if not os.path.exists( path + "/trainResized" ):
#    os.makedirs( path + "/trainResized" )
#if not os.path.exists( path + "/testResized" ):
#    os.makedirs( path + "/testResized" )

In [0]:
filepath = "https://github.com/Raginii/Color-Concept-Associations-using-Google-Images/blob/master/dataTop50.xlsx"
#"/content/drive/My Drive/Colab Notebooks/dataTop50.xlsx"
filepath = "dataTop50.xlsx"
df = pd.read_excel(filepath)
Q = df.as_matrix()
X = np.copy(Q[:,:30])
#xTemp =  np.copy(Q)
strtInd = [0, 2900, 5800, 8700, 11600, 14500, 17400, 20300, 23200, 26100, 29000, 31900, 34800]

In [9]:
X.shape

(34800, 30)

In [10]:
y = np.copy(Q[:,-1])
y.shape

(34800,)

In [0]:
X = np.column_stack([X,y])


In [12]:
X.shape

(34800, 31)

In [0]:
n = 3
from itertools import combinations 
  
# Get all combinations of length n
ftList = list(combinations(range(30), n))

In [14]:
len(ftList)

4060

In [0]:
def runRegressionSelFt(selFt):
    #print("Inside function")
    leastErr = 100
    #minErrFeature = None

    xNew = np.column_stack([X[:,selFt],X[:,-1]])

    # Linear Regression using Cross Validation

    # Leave one out cross validation -- strtInd holds the starting indices of set of fruit data 

    #print("\nSelected Features : ",selFt)
    sumErr = 0.0
    for i in range(len(strtInd)-1):
        xTemp = np.copy(xNew)
        # Leave one entire fruit group
        testFruit =  allConcepts[i]
        #print("\nLeaving out fruit group : ",allConcepts[i])
        testTemp = np.copy(xTemp[strtInd[i]:strtInd[i+1],:]) # Get all rows from the LOO group
        trainTemp = np.delete(xTemp,range(strtInd[i],strtInd[i+1]),0)
        #np.random.shuffle(trainTemp)
        row = len(trainTemp)
        #Training set
        X_train = np.column_stack([np.ones((row,1)), trainTemp[:,:-1]]) # add offset
        y_train = np.copy(trainTemp[:,-1])
        # Test set (hold out set)
        row2 = len(testTemp)
        X_test  = np.column_stack([np.ones((row2,1)), testTemp[:,:-1]]) #xTemp[row:tot,0:3]#
        y_test  = np.copy(testTemp[:,-1])


        # Create linear regression object
        regr = linear_model.LinearRegression(fit_intercept=False)
        #regr = linear_model.Lasso(alpha = 0.001,fit_intercept=False)
        #regr = linear_model.Ridge(alpha = 0.001,fit_intercept=False)

        # Train the model using the training sets
        regr.fit(X_train, y_train)

        # Make predictions using the testing set
        y_pred = regr.predict(X_test)
#         print("Score is : %.2f" % regr.score(X_test, y_test))
#         # The coefficients
#         print('Coefficients: ', regr.coef_)
#         #print('Intercept: \n', regr.intercept_)
#         # The mean squared error
#         print("\nMean squared error: %.4f"
#               % mean_squared_error(y_test, y_pred))
#         # Explained variance score: 1 is perfect prediction
#         print('Variance score: %.2f' % r2_score(y_test, y_pred))


        ############## For test purposes - Average prediction over all Images of LOO group ############
        totalTestImages = y_pred.size/58  #58N rows -> every image has 58 rows of data
        avg_pred = [np.sum([y_pred[p] for p in range(q,y_pred.size,58)])/totalTestImages for q in range(58)]

        # For Pearson correlation coefficient and the p-value (between avg_pred and true rating for fruit group)
        from scipy import stats
        trueRating = np.copy(y_test[0:58])

        sqErr = np.sum(pow(avg_pred - trueRating,2))
        #print("The true squared error (in color ratings) when leaving out",testFruit,"is: %.4f" % sqErr)
        sumErr = sumErr + sqErr

        corrCoeff, pVal = stats.pearsonr(avg_pred, trueRating)
    #     print("\nPearson’s correlation coefficient (between true ratings and avg predicted ratings) is %.4f" %corrCoeff)
    #     print("p-value %.4f" %pVal)

    #print("The average squared error of color ratings over all left out fruit categories for the selected feature is %.4f" % (sumErr/12))  
    if leastErr > sumErr:
        leastErr = sumErr
        minErrFeature = selFt
    #print("\nSelected Features : ",selFt,"Average Error : ", sumErr/12)
    #print("Exiting")
    return ([selFt,  sumErr/12 ])

In [16]:
pool = mp.Pool(mp.cpu_count())
print(pool)
results = pool.map(runRegressionSelFt, [selFt for selFt in ftList])

pool.close()
print(results)

[[(0, 1, 2), 1.483345807010832], [(0, 1, 3), 1.3250066507763274], [(0, 1, 4), 1.2542109462498117], [(0, 1, 5), 1.702359974496404], [(0, 1, 6), 1.7028416293924378], [(0, 1, 7), 1.5116903600008593], [(0, 1, 8), 1.358825885385917], [(0, 1, 9), 1.3033951421300702], [(0, 1, 10), 1.70089369251593], [(0, 1, 11), 1.6988768707118702], [(0, 1, 12), 1.5800252925886085], [(0, 1, 13), 1.4402310812182844], [(0, 1, 14), 1.396514318734985], [(0, 1, 15), 1.6998667945599786], [(0, 1, 16), 1.6952738663931985], [(0, 1, 17), 1.6426004137376424], [(0, 1, 18), 1.529967694543492], [(0, 1, 19), 1.4965034422733445], [(0, 1, 20), 1.699236516171929], [(0, 1, 21), 1.6932866542165896], [(0, 1, 22), 1.67582205114471], [(0, 1, 23), 1.593393922783507], [(0, 1, 24), 1.5675240924727687], [(0, 1, 25), 1.7026545180225137], [(0, 1, 26), 1.6941843201978701], [(0, 1, 27), 1.5107387996533452], [(0, 1, 28), 1.3615199208022835], [(0, 1, 29), 1.2993039054089537], [(0, 2, 3), 1.3251899236525413], [(0, 2, 4), 1.250863530553575], [

In [0]:
t = results

In [28]:
tval = [i for j,i in t]
len(tval)

4060

In [0]:
minInd = tval.index(min(tval))

In [33]:
t[minInd]

[(2, 14, 19), 1.194327359187234]